# Main Module
    # Call all the functions from other modules here to run the final output


In [ ]:
Def SMAs_Function(combined_values)

In [ ]:
    # Imports
    from pathlib import Path
    import hvplot.pandas
    import matplotlib.pyplot as plt

    from sklearn import svm
    from sklearn.preprocessing import StandardScaler
    from pandas.tseries.offsets import DateOffset
    from sklearn.metrics import classification_report
    # this helps import the dataframe from API Data Calls.ipynb, you need to run the API notebook to get this dataframe
    %store -r combined_values

# All Imports

In [ ]:
!pip install ipynb

In [ ]:
#Import Nasdaq Data Link (Quandl). Use "pip install nasdaq-data-link" to install library
import numpy as np
import pandas as pd
from urllib.request import urlopen
import json
from functools import reduce
import yfinance as yf
    

In [ ]:
# Imports
from pathlib import Path
import hvplot.pandas
import matplotlib.pyplot as plt

# Import StandardScaler and DateOffset
from sklearn.preprocessing import StandardScaler
from pandas.tseries.offsets import DateOffset

# Import classification_report
from sklearn.metrics import classification_report

# Import a new classifier from SKLearn
from sklearn import svm
from sklearn.linear_model import LogisticRegression

# this helps import the dataframe from API Data Calls.ipynb, you need to run the API notebook to get this dataframe
%store -r combined_values

In [ ]:
# Import Functions
#from ipynb.fs.full.Module1_API_Data_Calls.ipynb import data_import_cleanup_function

#from ipynb.fs.full.Module2_SMA.ipynb import SMAs_Function

#from ipynb.fs.full.Module3_Classifier_Module import classifier_function

import Functions_1_2_3_Combined.py

# Call Functions

In [ ]:
%run Functions
combined_values = data_import_cleanup_function()

In [ ]:
#%run 2_SMAs.ipynb 
X_train_scaled, y_train, y_test = SMAs_Function(combined_values, short_window = 20, long_window = 100)

In [ ]:
#%run 3_Classifier_Module.ipynb 
model_testing_report, cumulative_return_plot = classifier_function()

    ### Step 1: Import the dataset into a Pandas DataFrame.

In [ ]:
    # Filter the date index and close columns
    signals_df = combined_values.loc[:, ["Close"] ["value"]]

    # Use the pct_change function to generate  returns from close prices
    signals_df["Actual Returns"] = signals_df["Close"].pct_change()

    # Drop all NaN values from the DataFrame
    signals_df = signals_df.dropna()

    # Review the DataFrame
    # display(signals_df.head())
    # display(signals_df.tail())

In [ ]:
    ## Step 2: Generate trading signals using short- and long-window SMA values. 

In [ ]:
    # Set the short window and long window
    short_window = 20
    long_window = 100

    # Generate the fast and slow simple moving averages (4 and 100 days, respectively)
    signals_df['SMA_Fast'] = signals_df['Close'].rolling(window=short_window).mean()
    signals_df['SMA_Slow'] = signals_df['Close'].rolling(window=long_window).mean()

    signals_df = signals_df.dropna()

    # Review the DataFrame
    display(signals_df.head())
    display(signals_df.tail())

In [ ]:
    # Initialize the new Signal column
    signals_df['Signal'] = 0.0

    # When Actual Returns are greater than or equal to 0, generate signal to buy stock long
    signals_df.loc[(signals_df['Actual Returns'] >= 0), 'Signal'] = 1

    # When Actual Returns are less than 0, generate signal to sell stock short
    signals_df.loc[(signals_df['Actual Returns'] < 0), 'Signal'] = -1

    # Review the DataFrame
    display(signals_df.head())
    display(signals_df.tail())

In [ ]:
    signals_df['Signal'].value_counts()

In [ ]:
    # Calculate the strategy returns and add them to the signals_df DataFrame
    signals_df['Strategy Returns'] = signals_df['Actual Returns'] * signals_df['Signal'].shift()

    # Review the DataFrame
    display(signals_df.head())
    display(signals_df.tail())

In [ ]:
    # Plot Strategy Returns to examine performance
    (1 + signals_df['Strategy Returns']).cumprod().plot()

In [ ]:
    ### Step 3: Split the data into training and testing datasets.

In [ ]:
    # Assign a copy of the sma_fast and sma_slow columns to a features DataFrame called X
    X = signals_df[['SMA_Fast', 'SMA_Slow']].shift().dropna()

    # Review the DataFrame
    X.head()

In [ ]:
    # Create the target set selecting the Signal column and assiging it to y
    y = signals_df['Signal']

    # Review the value counts
    y.value_counts()

In [ ]:
    # Select the start of the training period
    training_begin = X.index.min()

    # Display the training begin date
    print(training_begin)

In [ ]:
    # Select the ending period for the training data with an offset of 3 months
    training_end = X.index.min() + DateOffset(months=3)

    # Display the training end date
    print(training_end)

In [ ]:
    # Generate the X_train and y_train DataFrames
    X_train = X.loc[training_begin:training_end]
    y_train = y.loc[training_begin:training_end]

    # Review the X_train DataFrame
    X_train.head()

In [ ]:
    # Generate the X_test and y_test DataFrames
    X_test = X.loc[training_end+DateOffset(hours=1):]
    y_test = y.loc[training_end+DateOffset(hours=1):]

    # Review the X_test DataFrame
    X_train.head()

In [ ]:
    # Scale the features DataFrames

    # Create a StandardScaler instance
    scaler = StandardScaler()

    # Apply the scaler model to fit the X-train data
    X_scaler = scaler.fit(X_train)

    # Transform the X_train and X_test DataFrames using the X_scaler
    X_train_scaled = X_scaler.transform(X_train)
    X_test_scaled = X_scaler.transform(X_test)

In [ ]:
    return  X_train_scaled, y_train, y_test